In [2]:
import sys
sys.path.append('..')
from seismic_postprocessing import *

data_path = 'data/'
min_coverage = 3000

seismic_df = read_seismic_output(output_path=os.path.join(data_path, 'out'), ref2plate=None, min_cov=min_coverage)

Number of references in output:  844


In [3]:
filtered_df = filter_bad_references(seismic_df, min_reads=min_coverage, signal_tresholds=(0.3, 0.7), min_covered=0.5, primers=None)
plot_signal_histogram(filtered_df)

-> Before filtering: Number of sequences: 844 and number of unique sequences: 463
-> After filtering low reads: Number of sequences: 227 and number of unique sequences: 137
-> After filtering high mutations: Number of sequences: 225 and number of unique sequences: 136
(0 references were updated and 1 references were removed)
-> After filtering not well covered: Number of sequences: 116 and number of unique sequences: 64


In [9]:
normalized_df = normalize_dms(filtered_df, percentile=95)
print_summary(normalized_df)
plot_histogram(normalized_df)

0 :  116
Total:  116


In [11]:
combined_df = combine_replicates(normalized_df, min_correlation=0.8)
print_summary(combined_df)

0 :  62
Total:  62


In [12]:
# Output data
output_path = os.path.join(data_path, 'processed')
if os.path.exists(output_path):
    os.system('rm -r {}'.format(output_path))
    
plot_dms_signal(combined_df, output_path)
dump_df_to_dreem_format(combined_df, name = data_path.split('/')[-2], path=output_path)

# RouskinHF

In [15]:
import envbash
envbash.load.load_envbash('../.env')
from rouskinhf import convert, upload_dataset, get_dataset

In [14]:
data = convert(
    'seismic',
    'data/processed/data.json',
    name='human_mRNA',
    path_out='data',
    filter=True,
    predict_structure=True
)

Parsing dreem output file: 100%|██████████| 62/62 [00:12<00:00,  4.95it/s]


Over a total of 62 datapoints, there are:
### OUTPUT
- 56 valid datapoints
### MODIFIED
- 0 multiple sequences with the same reference (renamed reference)
- 0 duplicate sequences with different structure / dms / shape
### FILTERED OUT
- 0 invalid datapoints (ex: sequence with non-regular characters)
- 0 datapoints with bad structures
- 0 duplicate sequences with the same structure / dms / shape
- 6 datapoints removed because of low AUROC (<0.8)


## To HuggingFace

In [16]:
upload_dataset(
    'data/human_mRNA/data.json',
    commit_message='first commit with new pipeline',
    exist_ok=True,
)